In [1]:
import pandas as pd
import numpy as np
import sys
import os
import cv2

import matplotlib.pyplot as plt

from skimage import io,filters

In [2]:
IMAGES_DIR = "./datasets/Plate_Image"
GRAYED_DIR = "./outputs/Grayed_Image"
LABELS_FILE = "./datasets/Plate_Index.xls"

In [3]:
xls_file = pd.ExcelFile(LABELS_FILE)
xls_file.sheet_names

['Sheet1', 'Sheet2', 'Sheet3']

In [4]:
data = xls_file.parse("Sheet1")
data.columns = ['Image ID', 'Corner1_x', 'Corner1_y', 'Corner2_x', 'Corner2_y', 'Corner3_x', 'Corner3_y', 
                'Corner4_x', 'Corner4_y', 'Char1', 'Char2', 'Char3','Char4', 'Char5', 'Char6', 'Char7']

In [5]:
data.head()

,Image ID,Corner1_x,Corner1_y,Corner2_x,Corner2_y,Corner3_x,Corner3_y,Corner4_x,Corner4_y,Char1,Char2,Char3,Char4,Char5,Char6,Char7
0,702172519671,336,323,688,310,682,412,339,430,10,15,12,8,5,5,0
1,702172544593,301,286,728,291,725,423,306,418,10,23,26,1,9,6,9
2,702172623671,294,299,684,322,675,411,299,392,10,20,21,8,3,9,9
3,702172729578,200,267,723,291,707,398,222,381,10,21,22,7,9,2,5
4,702172759468,348,223,608,278,599,359,346,305,10,21,29,2,0,1,1


In [6]:
fnames = [os.path.join(IMAGES_DIR, str(fname) + ".jpg") for fname in data["Image ID"]]

In [8]:
for i in range(len(fnames)):
    x1, y1 = data["Corner1_x"][i], data["Corner1_y"][i]   
    x2, y2 = data["Corner2_x"][i], data["Corner2_y"][i]
    x3, y3 = data["Corner3_x"][i], data["Corner3_y"][i]
    x4, y4 = data["Corner4_x"][i], data["Corner4_y"][i]
    
    plate_x_l = min(x1, x4)
    plate_x_r = max(x2, x3)
    plate_y_u = min(y1, y2)
    plate_y_d = max(y3, y4)
    
    width = (np.sqrt(np.square(x1-x2) + np.square(y1-y2)) + \
             np.sqrt(np.square(x3-x4) + np.square(y3-y4))) / 2 
    height = (np.sqrt(np.square(x1-x4) + np.square(y1-y4)) + \
             np.sqrt(np.square(x2-x3) + np.square(y2-y3))) / 2 
    
    path = fnames[i]
    img = io.imread(path, as_gray=True)

    rows = plate_x_r - plate_x_l
    cols = plate_y_d - plate_y_u
    src = np.float32([[x1, y1], [x2, y2], [x4, y4], [x3, y3]])
    dst = np.float32([[0,0], [width,0], [0, height], [width, height]])
    # 求得透视投影变换矩阵
    M_perspective = cv2.getPerspectiveTransform(src,dst)
    # 执行透视投影变换,并resize
    cropImg = cv2.warpPerspective(img, M_perspective, (int(width), int(height)))
    
    thresh = filters.threshold_otsu(cropImg)
    dst = (cropImg > thresh) * 1
    if not os.path.exists(GRAYED_DIR):
        os.mkdir(GRAYED_DIR)     
    path_out = os.path.join(GRAYED_DIR, "gray_"+str(i+1)+".png")
    dst = dst.astype(np.uint8) * 255
    io.imsave(path_out, dst)

## PreProcessToCorrect

In [41]:
for i in range(147, 148):
    x1, y1 = data["Corner1_x"][i], data["Corner1_y"][i]   
    x2, y2 = data["Corner2_x"][i], data["Corner2_y"][i]
    x3, y3 = data["Corner3_x"][i], data["Corner3_y"][i]
    x4, y4 = data["Corner4_x"][i], data["Corner4_y"][i]
    
    plate_x_l = min(x1, x4)
    plate_x_r = max(x2, x3)
    plate_y_u = min(y1, y2)
    plate_y_d = max(y3, y4)
    
    width = (np.sqrt(np.square(x1-x2) + np.square(y1-y2)) + \
             np.sqrt(np.square(x3-x4) + np.square(y3-y4))) / 2 
    height = (np.sqrt(np.square(x1-x4) + np.square(y1-y4)) + \
             np.sqrt(np.square(x2-x3) + np.square(y2-y3))) / 2 
    
    path = fnames[i]
    print(path)
    img = io.imread(path, as_gray=True)

    rows = plate_x_r - plate_x_l
    cols = plate_y_d - plate_y_u
    src = np.float32([[x1, y1], [x2, y2], [x4, y4], [x3, y3]])
    dst = np.float32([[0,0], [width,0], [0, height], [width, height]])
    # 求得透视投影变换矩阵
    M_perspective = cv2.getPerspectiveTransform(src,dst)
    # 执行透视投影变换,并resize
    cropImg = cv2.warpPerspective(img, M_perspective, (int(width), int(height)))
    
    thresh = filters.threshold_otsu(cropImg)
    dst = ((cropImg > thresh*1.23) * 1).astype(np.uint8) * 255
    
    if not os.path.exists(GRAYED_DIR):
        os.mkdir(GRAYED_DIR)     
    path_out = os.path.join(GRAYED_DIR, "gray_"+str(i+1)+".png")
    io.imsave(path_out, dst)

./datasets/Plate_Image/704012523468.jpg
